In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
import sys
sys.path.append('../pipeline/src')

import pandas as pd
import numpy as np
from datamodules.paired_dataset import PairedDataset
from datamodules.paired_datamodule import PairedProteinDataModule, PairedProteinDataModuleConfig
from datamodules.datamodule import ProteinDataModuleConfig, SamplingConfig, ProteinDataModule


In [12]:
# Create paired dataset from DataFrames
import pandas as pd

train_dataset_path = '../pipeline/output/datasets/train.pt'
val_dataset_path = '../pipeline/output/datasets/val.pt'
test_dataset_path = '../pipeline/output/datasets/test.pt'
similarity_file_path = '../pipeline/output/mmseqs/all_seqs_similarity.tsv'
cluster_path = '../pipeline/output/mmseqs/all_seqs_clust.tsv'


In [23]:

sampling_conf = SamplingConfig(
    cluster_path=cluster_path,
    train_n_pdb=1.0,
    train_intersect_val_clusters=False,
    train_intersect_test_clusters=False,
    val_n_pdb=1.0,
    test_n_pdb=1.0,
    train_max_chains_per_pdb=1,
    val_max_chains_per_pdb=1,
    test_max_chains_per_pdb=1,
)
paired_datamodule_config = PairedProteinDataModuleConfig(
    _target_="pipeline.src.datamodules.paired_datamodule.PairedProteinDataModuleConfig",
    name="paired_datamodule",
    train_dataset_path=train_dataset_path,
    val_dataset_path=val_dataset_path,
    test_dataset_path=test_dataset_path,
    tokenizer_name="facebook/esm2_t33_650M_UR50D",
    batch_size=1,
    num_workers=1,
    max_seq_length=1024,
    contact_threshold=8.0,
    sampler=sampling_conf,
    similarity_file_path=similarity_file_path,
    min_similarity_threshold=0.3,
)

paired_datamodule = PairedProteinDataModule(paired_datamodule_config)
paired_datamodule.setup()




INFO:datamodules.paired_datamodule:Num train in queries: 16015
INFO:datamodules.paired_datamodule:Num test in queries: 586
INFO:datamodules.paired_datamodule:Loaded 16601 similarity pairs: 
INFO:datamodules.paired_datamodule:                 target_id  identity
query_id                            
train_1UVY_A  train_3AQ9_B     0.390
train_5O98_B  train_5LCX_A     0.556
train_3UPC_D  train_1T2J_A     0.841
train_2XYU_A  train_6Q7G_A     0.759
train_4TYO_A  train_3I6C_A     0.936
train_6JOM_A  train_5ICH_A     0.373
train_6T0Z_B  train_6SZ8_A     0.931
train_5P0Y_A  train_5R33_A     0.918
test_5WM4_A   train_7TYB_A     0.550
train_7K9U_B  train_7K9R_B     0.930
INFO:datamodules.datamodule:Sample Train: After removing intersection with val clusters, train dataset size: 21676
INFO:datamodules.datamodule:Sample Train: After removing intersection with test clusters, train dataset size: 17575
INFO:datamodules.datamodule:Sample Train: After selecting 1.0 n/frac of pdbs, train dataset size: 17

             id pdb_id chain_id  length  \
0  train_106M_A   106M        A     154   
1  train_109L_A   109L        A     162   
2  train_111M_A   111M        A     154   
3  train_115L_A   115L        A     162   
4  train_122L_A   122L        A     162   

                                     residue_indices  mean_b_factor  
0  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...      16.246299  
1  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...      14.398765  
2  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...      20.670714  
3  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...      13.986358  
4  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...      14.248951  
             id pdb_id chain_id  length  \
0  train_1A77_A   1A77        A     315   
1  train_1AQJ_A   1AQJ        A     381   
2  train_1AQJ_B   1AQJ        B     383   
3  train_1C1K_A   1C1K        A     217   
4  train_1C90_A   1C90        A     265   

                                     residue_indices  mean_b_f

/workspace/esm-2-contact-map/ipynb/../pipeline/src/datamodules/datamodule.py:63: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_pdb = df.groupby("pdb_id").apply(
INFO:datamodules.datamodule:Sample Train: After selecting 1 chains per pdb, train dataset size: 10265
INFO:datamodules.datamodule:Sample Train: After sampling, train dataset size: 10265
INFO:datamodules.datamodule:Sample Val: After selecting 1.0 n/frac of pdbs, val dataset size: 767
/workspace/esm-2-contact-map/ipynb/../pipeline/src/datamodules/datamodule.py:63: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the

In [36]:
for batch in paired_datamodule.train_dataloader():
    print("OUR BATCH")
    print(batch)
    break

[{'primary_sequence': {'sequence': 'AAYQVLIVGAGFSGAETAFWLAQKGVRVGLLTQSLDAVMMPFLPPKPPFPPGSLLERAYDPKDERVWAFHARAKYLLEGLRPLHLFQATATGLLLEGNRVVGVRTWEGPPARGEKVVLAVGSFLGARLFLGGVVEEAGRLSEASYPDLLEDLSRLGFRFVEREGEVPPGYRVRYLAFHPEEWEEKTFRLKRLEGLYAVGLCVREGDYARMSEEGKRLAEHLLHEL', 'seq_length': 225, 'metadata': {'id': 'train_2CUL_A', 'pdb_id': '2CUL', 'chain_id': 'A', 'length': 225, 'residue_indices': array([  2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,  14,
        15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
        28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,
        41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,
        54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,
        67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,  79,
        80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,  92,
        93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104, 105,
       106, 107, 108, 109, 110, 111, 